# How to train a new language model from scratch using Transformers and Tokenizers Sagemaker
* https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb

## Setup
First install `sagemaker` and `sagemaker[local]1. Also install the latest transformers from HuggingFace.

In [1]:
!pip install -q sagemaker sagemaker[local]

# Install `transformers` from master
!pip install -q git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers|sagemaker'

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
sagemaker           1.60.2    
tokenizers          0.7.0     
transformers        2.11.0    
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Open session and bucket
Open sagemaker session and setup the bucket for data upload

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/hunkim-transformer'

### Set a role
You need a role that can execute sagemaker and read/write S3. Create a role in your IAM setting, and use the role name 

In [3]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='hunkimSagemaker')['Role']['Arn']
    
print(role)

arn:aws:iam::294038372338:role/hunkimSagemaker


### Data downlaod
In this example, we are using a tinyshakespeare.

In [4]:
# in this notebook we'll only get one of the files (the Oscar one) for the sake of simplicity and performance
!mkdir -p data
!wget -O data/oscar.eo.txt -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

--2020-06-11 10:53:28--  https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 52.85.193.66, 52.85.193.97, 52.85.193.210, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|52.85.193.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312733741 (298M) [text/plain]
Saving to: ‘data/oscar.eo.txt’

data/oscar.eo.txt   100%[===================>] 298.25M   846KB/s    in 13m 11s 

2020-06-11 11:06:38 (386 KB/s) - ‘data/oscar.eo.txt’ saved [312733741/312733741]



### Uplaod data to S3
Upload the data to S3 and the S3 path will be passed to the train

In [5]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-transformer


## Train
First we need to have a code directory which includes requrements.txt and train/infer code. Then, we will create pytorch container in the Sagemaker and execute the train. Finally, the train program will save models and necessary data in the `model_dir` and upload it to S3.

Later the uploaded model in S3 will be used for model serving (endpoint)

In [6]:
!pygmentize code/requirements.txt

transformers == 2.11.0
boto3


In [7]:
!pygmentize code/train.py

from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

# Based on github.com/pytorch/examples/blob/master/word_language_model
import argparse
import math
import os
from shutil import copy
import time
import torch
import torch.nn as nn

# Run outside of SM for quick testing
if 'SM_MODEL_DIR' not in os.environ:
    print("Running locally?")

    os.environ['SM_MODEL_DIR'] = "./.model_dir"
    os.environ['SM_OUTPUT_DATA_DIR'] = "./.out_data"
    os.environ['SM_CHANNEL_TRAINING']="./.train_data"

    # Make a sample data 
    os.makedirs('./.model_dir', exist_ok=True)
    os.makedirs('./.out_data', exist_ok=True)
    os.makedirs('./.train_data', exist_ok=True)

    #f = open(os.path.join(os.environ['SM_CHANNEL_TRAINING'],"sample.txt"), "a")
    #f.write("Now the file has more content!\nThis is cool\n")
    #f.close()


parser = argparse.ArgumentParser(description='T

### PyTorch Container
define the container (framework) version and other options including hyperparameters. For local testing, use the `local` or `local_gpu` for the train_instance_type.

specify the `source_dir` and `entry_pint` to point the train code.

In [8]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.5.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    #train_instance_type='local',
                    source_dir='code',
                    hyperparameters={
                        "vocab-size": 52_000,
                        "max-position-embeddings": 514,
                        "num-attention-heads": 12,
                        "num-hidden-layers": 6,
                        "type-vocab-size": 1,
                        "overwrite-output-dir": True,
                        "num-train-epochs": 1,
                        "per-gpu-train-batch-size": 64,
                        "save-steps": 10_000,
                        "save-total-limit": 2,
                        "token-max-len": 512
                    })

### Fire training
`fit` will sectue the train code we specified. Make sure we pass the training data

In [9]:
%%time
estimator.fit({'training': inputs})

gz",
    "module_name": "train",
    "network_interface_name": "eth0",
    "num_cpus": 4,
    "num_gpus": 1,
    "output_data_dir": "/opt/ml/output/data",
    "output_dir": "/opt/ml/output",
    "output_intermediate_dir": "/opt/ml/output/intermediate",
    "resource_config": {
        "current_host": "algo-1",
        "hosts": [
            "algo-1"
        ],
        "network_interface_name": "eth0"
    },
    "user_entry_point": "train.py"
}

Environment variables:

SM_HOSTS=["algo-1"]
SM_NETWORK_INTERFACE_NAME=eth0
SM_HPS={"max-position-embeddings":514,"num-attention-heads":12,"num-hidden-layers":6,"num-train-epochs":1,"overwrite-output-dir":true,"per-gpu-train-batch-size":64,"save-steps":10000,"save-total-limit":2,"token-max-len":512,"type-vocab-size":1,"vocab-size":52000}
SM_USER_ENTRY_POINT=train.py
SM_FRAMEWORK_PARAMS={}
SM_RESOURCE_CONFIG={"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}
SM_INPUT_DATA_CONFIG={"training":{"RecordWrapperType":"None","S3

## Serving
The first step of serving is getting the models stored in S3 and launch a sagemaker serving container. 

### S3 models
Makr sure the S3 file has models and necessaty files.

In [10]:
training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
print(trained_model_location)
# s3://sagemaker-us-west-2-294038372338/pytorch-training-2020-06-10-23-53-28-771/output/model.tar.gz

s3://sagemaker-us-west-2-294038372338/pytorch-training-2020-06-11-02-07-01-602/output/model.tar.gz


### Json serializer
We are passing our data to json.

In [11]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

### Infer code
First we will load our model using `model_fn`. Then using the loaded model, we implement `predict_fn`. The `input_fn` and `output_fn` will handle the input/out data checking and processing. 

In [12]:
!pygmentize code/infer.py

import json
import logging
import os

import torch

from transformers import pipeline


JSON_CONTENT_TYPE = 'application/json'

logger = logging.getLogger(__name__)


def model_fn(model_dir):
    logger.info('Loading the model.')

    fill_mask = pipeline(
        "fill-mask",
        model=model_dir,
        tokenizer=model_dir,
    )
    return fill_mask


def input_fn(serialized_input_data, content_type=JSON_CONTENT_TYPE):
    logger.info('Deserializing the input data.')
    if content_type == JSON_CONTENT_TYPE:
        input_data = json.loads(serialized_input_data)
        if 'text' not in input_data:
            raise Exception('\'text\' has to be set.')
        return input_data
    raise Exception('Requested unsupported ContentType in content_type: ' + content_type)


def output_fn(prediction_output, accept=JSON_CONTENT_TYPE):
    logger.info('Serializing the generated output.')
    if accept == JSON_CONTENT_TYPE:
        return json.dumps(prediction_output), accept
    raise Ex

### Config Serving Contaner
Create a container using the stored model (in S3). Specify the infer code and source directory.


In [13]:
from sagemaker.pytorch import PyTorchModel

training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']

model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.5.0',
                     entry_point='infer.py',
                     source_dir='code',
                     predictor_cls=JSONPredictor)

### Deploy!
Finally, it's ready to deploy. Let's fire it! The endpoint address will be used to connect to the server.

In [14]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Get the end point
endpoint = predictor.endpoint
print(endpoint)

---------------!pytorch-inference-2020-06-11-02-22-03-485


## Testing
Finally we can test the endpoint. First, we will just reuse the predictor for a quick test.

In [15]:
input = {
    'text': "We are <mask>."
}
response = predictor.predict(input)
print(response)

[{'sequence': '<s> We are,.</s>', 'score': 0.061813835054636, 'token': 16}, {'sequence': '<s> We are the.</s>', 'score': 0.02348240092396736, 'token': 1682}, {'sequence': '<s> We are I.</s>', 'score': 0.01902938261628151, 'token': 413}, {'sequence': '<s> We are to.</s>', 'score': 0.013471029698848724, 'token': 1211}, {'sequence': '<s> We are a.</s>', 'score': 0.013361896388232708, 'token': 278}]


### Remote
This example shows how to connect to the endpoint remotely.

In [16]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

input = {
    'text': "we are <mask>."
}
payload = json.dumps(input)

response = client.invoke_endpoint(
    EndpointName=endpoint, 
    ContentType="application/json",
    Accept="application/json" ,
    Body=payload
)

print(response['Body'].read())  

b'[{"sequence": "<s> we are,.</s>", "score": 0.060730352997779846, "token": 16}, {"sequence": "<s> we are the.</s>", "score": 0.023240746930241585, "token": 1682}, {"sequence": "<s> we are I.</s>", "score": 0.019465802237391472, "token": 413}, {"sequence": "<s> we are to.</s>", "score": 0.013306406326591969, "token": 1211}, {"sequence": "<s> we are a.</s>", "score": 0.013236642815172672, "token": 278}]'


## Shutdown
After finishing, delete the endpoint.

In [17]:
sagemaker_session.delete_endpoint(predictor.endpoint)